<a href="https://colab.research.google.com/github/freedomtan/clip_score_on_android/blob/main/verify_text_encoder_fp16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q diffusers
!pip install -q accelerate
!pip install -q keras_cv

In [2]:
from diffusers import StableDiffusionPipeline
import torch

import keras_cv
import keras

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
# pipe = StableDiffusionPipeline.from_pretrained(model_id)
pipe = pipe.to("cuda")

torch_text_encoder = pipe.text_encoder

keras.mixed_precision.set_global_policy("mixed_float16")
pipeline = keras_cv.models.StableDiffusion(
    img_width=512, img_height=512, jit_compile=True
)

Using TensorFlow backend


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [3]:
first_10_prompts = [
    "A city at night with people walking around.",
    "The large clock was prominently displaying the time.",
    "A person in winter gear riding a snowboard.",
    "A small bird is perched on an empty bird feeder.",
    "Baseballs players sliding to base and jumping during the game.",
    "A woman playing tennis in a white outfit",
    "A woman looking at her phone next to a car being towed.",
    "A man in a wheelchair and another sitting on a bench that is overlooking the water.",
    "Rows of motor bikes and helmets in a city"
]

In [4]:
for p in first_10_prompts:
  ids = pipe.tokenizer.encode(p)
  id_len = len(ids)
  ids_padded = ids + ([49407] * (77 - id_len))

  token = torch.unsqueeze(torch.tensor(ids_padded).to("cuda"), 0)
  torch_output = torch_text_encoder(input_ids=token).last_hidden_state.cpu().detach().numpy()
  tf_output = pipeline.encode_text(p)
  print(torch_output - tf_output)

[[[ 0.000e+00  4.578e-05  3.052e-05 ... -2.441e-04  0.000e+00 -6.104e-05]
  [ 1.144e-03  9.766e-04  2.441e-04 ...  4.883e-04  4.883e-04  0.000e+00]
  [ 9.766e-04 -2.930e-03  3.906e-03 ... -1.953e-03 -1.465e-03 -1.953e-03]
  ...
  [-2.930e-03 -4.883e-04  1.465e-03 ...  1.221e-03 -4.883e-04 -4.883e-04]
  [-1.465e-03 -1.465e-03 -1.343e-03 ...  1.709e-03 -1.221e-03  1.953e-03]
  [-2.441e-03 -4.883e-04 -3.418e-03 ...  1.709e-03  0.000e+00 -1.465e-03]]]
[[[ 0.000e+00  4.578e-05  3.052e-05 ... -2.441e-04  0.000e+00 -6.104e-05]
  [ 1.282e-03 -4.883e-04  0.000e+00 ... -4.883e-04  3.906e-03 -4.883e-04]
  [ 9.766e-04 -4.883e-04  2.441e-03 ... -1.465e-03  0.000e+00  2.930e-03]
  ...
  [-2.197e-03 -4.883e-04 -1.465e-03 ...  0.000e+00 -2.930e-03  1.221e-04]
  [-2.930e-03 -2.441e-03 -1.465e-03 ... -9.766e-04  0.000e+00  6.104e-04]
  [-1.709e-03  1.465e-03 -9.766e-04 ... -9.766e-04 -9.766e-04  2.441e-03]]]
[[[ 0.000e+00  4.578e-05  3.052e-05 ... -2.441e-04  0.000e+00 -6.104e-05]
  [ 1.144e-03  9.766e-